In [57]:
from dotenv import load_dotenv; load_dotenv()

True

In [11]:
# Get Data
#import csv


from datasets import load_dataset
ds = load_dataset("stanfordnlp/sentiment140")


Generating test split: 100%|██████████| 498/498 [00:00<00:00, 22408.02 examples/s]


In [23]:
docs = ds['train'][:2000]['text']

In [24]:
# Get our encoder to encode 
from sentence_transformers import SentenceTransformer 
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
data_emb = model.encode(docs) #16 seconds on M1 Mac 8gb


In [28]:
# Now set up the DB to acceept the data

import chromadb

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="SampleDB")

collection.add(
    embeddings=data_emb.tolist(),
    documents=docs,
    ids=[str(idx) for idx in range(len(data_emb))]) # Doc ID's are requried




In [42]:
#prep the question in the encoding space

question = 'What is the status of the chalkboard?'
question_emb = model.encode(question)

results = collection.query(query_embeddings=question_emb.tolist(), n_results=10)

context = ' '.join(results['documents'][0]) # Pulling out a lists of lists




In [63]:
# Call the models to derterim the answer by response

prompt = f'Given the following data, Please answer the question:  \n\n ##question \n {question}\n\n ##context \n {context}'


from aimodels.client import MultiFMClient
client = MultiFMClient()

messages = [
    {"role": "system", "content": "You are a helpful agent, who answers with brevity. "},
    {"role": "user", "content": prompt},
]


In [54]:
#groq_llama3_8b = "groq:llama3-8b-8192" 
response = client.chat.completions.create(model="groq:llama3-70b-8192", messages=messages)

print(response.choices[0].message.content)


The status of the chalkboard is: USELESS (because there is no chalk).


In [45]:
results['documents'][0][0]

"Damn... I don't have any chalk! MY CHALKBOARD IS USELESS "

In [64]:
client = MultiFMClient() #have to reinaiatize client or for the API key. 

response = client.chat.completions.create(model="anthropic:claude-3-opus-20240229", messages=messages)
print(response.choices[0].message.content)

Based on the context provided, the status of the chalkboard is useless because the person doesn't have any chalk.
